In [ ]:
import hmftpy as hmf
import numpy as np
import matplotlib.pyplot as plt
from hmftpy.plaquettes.square import plaq4
from hmftpy.plaquettes.triangular import plaq3, plaq12
from quspin.basis import spin_basis_1d
from quspin.operators import quantum_operator
from tqdm import tqdm

# Dzyaloshinskii–Moriya interaction
Here, we consider the DM interaction
\begin{align*}
    H_{DM} =&\vec D \cdot \sum_{\langle i, j\rangle} \vec \sigma_i \times\vec \sigma_j\\
    =&     D_x\sum_{\langle i, j \rangle}\left(\sigma_i^y\sigma_j^z - \sigma_i^z \sigma_j^y\right)
    +     D_y\sum_{\langle i, j \rangle} \left(\sigma_i^z\sigma_j^x - \sigma_i^x \sigma_j^z\right)
    +     D_z\sum_{\langle i, j \rangle} \left(\sigma_i^x\sigma_j^y - \sigma_i^y \sigma_j^x\right).
\end{align*}
For now, let's choose $D_z = 1$, $D_x=D_y=0$

In [ ]:
basis = spin_basis_1d(3, pauli=0) # pauli=0 fixes the operators as spin-1/2, rather than pauli matrices
interactions = {'nearest': {'xy': 1, 'yx': -1}}
Hi = hmf.operators.inner_hamiltonian(plaq3, interactions, basis, every_other=True, checks=True)
e, v = Hi.eigsh(k=1, which='SA')
ei = e[0]
Hp = hmf.operators.periodic_hamiltonian(plaq3, interactions, basis, every_other=True, checks=True)
e, v = Hp.eigsh(k=1, which='SA')
ep = e[0]
print('ED energy with OBC: {}'.format(ei))
print('ED energy with PBC: {}'.format(ep))

e_hmft, v, mf, cvg = hmf.do_hmft(plaq3, interactions, basis, every_other=True)

print('HMFT energy: {}'.format(e_hmft))
print('HMFT converged? {}'.format(cvg))

I want to check my work. By setting `every_other=True`, I restrict the sums to
\begin{equation}
\sum_i \sum_{j\in \mathcal N(i)} \rightarrow \sum_i \sum_{j\in \mathcal N(i), j < i}
\end{equation}
where $\mathcal N(i)$ is the list of neighbors of $i$. Without this restriction, the DMI terms cancel due to antisymmetry of the cross product.

On just the three-site triangle with open BC, the interaction in the first case (with `every_other=False`) is
\begin{equation}
(\sigma_0^x\sigma_1^y - \sigma_0^y \sigma_1^x)
+ (\sigma_0^x\sigma_2^y - \sigma_0^y \sigma_2^x)
+ (\sigma_1^x\sigma_0^y - \sigma_1^y \sigma_0^x)
+ (\sigma_1^x\sigma_2^y - \sigma_1^y \sigma_2^x)
+ (\sigma_2^x\sigma_0^y - \sigma_2^y \sigma_0^x)
+ (\sigma_2^x\sigma_1^y - \sigma_2^y \sigma_1^x)=0.
\end{equation}
With the restricted sum, I get
\begin{equation}
(\sigma_1^x\sigma_0^y - \sigma_1^y \sigma_0^x)
+ (\sigma_2^x\sigma_0^y - \sigma_2^y \sigma_0^x)
+ (\sigma_2^x\sigma_1^y - \sigma_2^y \sigma_1^x).
\end{equation}
If I instead sum bonds with some ordering, say $\{0,1\}$, $\{1,2\}$, $\{2,0\}$, I get
\begin{equation}
(\sigma_0^x\sigma_1^y - \sigma_0^y \sigma_1^x)
+ (\sigma_1^x\sigma_2^y - \sigma_1^y \sigma_2^x)
+ (\sigma_2^x\sigma_0^y - \sigma_2^y \sigma_0^x),
\end{equation}
which is not the same thing, since the terms are the same but the signs are not. What results do we get from this case?

In [ ]:
dm_lst = [['xy', [[1, 0, 1], [1, 1, 2], [1, 2, 0]]],
          ['yx', [[-1, 0, 1], [-1, 1, 2], [-1, 2, 0]]]
         ]
H_dm = quantum_operator({'static': dm_lst}, basis=basis)
dm_lst = [['xy', [[1, 0, 1], [1, 1, 2], [1, 2, 0]]],
          ['yx', [[-1, 0, 1], [-1, 1, 2], [-1, 2, 0]]]
         ]
e, v = H_dm.eigh()
print(e[0])

To fix this inconsistency, I added an additional way of forming Hamiltonians. For some clusters, I have added lists of nearest neighbor bonds with some consistently-chosen direction (counter-clockwise in upright triangles in these cases). This seems to give equivalent results to the previous case (skipping every-other bond)

In [ ]:
basis = spin_basis_1d(3, pauli=0) # pauli=0 fixes the operators as spin-1/2, rather than pauli matrices
interactions = {'x_bonds': {'xy': 1, 'yx': -1}, 'y_bonds': {'xy': 1, 'yx': -1}, 'z_bonds': {'xy': 1, 'yx': -1}}
Hi = hmf.operators.inner_hamiltonian(plaq3, interactions, basis, checks=True)
e, v = Hi.eigsh(k=1, which='SA')
ei = e[0]
Hp = hmf.operators.periodic_hamiltonian(plaq3, interactions, basis, checks=True)
e, v = Hp.eigsh(k=1, which='SA')
ep = e[0]
print('ED energy with OBC: {}'.format(ei))
print('ED energy with PBC: {}'.format(ep))

e_hmft, v, mf, cvg = hmf.do_hmft(plaq3, interactions, basis)

print('HMFT energy: {}'.format(e_hmft))
print('HMFT converged? {}'.format(cvg))

Now, let's apply this to a more useful (12-site) cluster:

In [ ]:
basis = spin_basis_1d(12, pauli=0) # pauli=0 fixes the operators as spin-1/2, rather than pauli matrices
interactions = {'x_bonds': {'xy': 1, 'yx': -1}, 'y_bonds': {'xy': 1, 'yx': -1}, 'z_bonds': {'xy': 1, 'yx': -1}}
Hi = hmf.operators.inner_hamiltonian(plaq12, interactions, basis, every_other=True, checks=True)
e, v = Hi.eigsh(k=1, which='SA')
ei = e[0]
Hp = hmf.operators.periodic_hamiltonian(plaq12, interactions, basis, every_other=True, checks=True)
e, v = Hp.eigsh(k=1, which='SA')
ep = e[0]
print('ED energy with OBC: {}'.format(ei))
print('ED energy with PBC: {}'.format(ep))

e_hmft, v, mf, cvg = hmf.do_hmft(plaq12, interactions, basis)

print('HMFT energy: {}'.format(e_hmft))
print('HMFT converged? {}'.format(cvg))

# Longer range DM interactions

Next-nearest neighbors on the triangular lattice form three disconnected, triangular sublattice that are rotated 60$^\circ$ from the underlying lattice. As with nearest neighbors, I will assume some ordering of bonds to get nonzero DM interactions, say CCW on triangles pointing right. 

That is to say, I want $D_{ij}$ to be positive when $j$ is either above and left, above and right, or directly below $i$, and negative otherwise. Note this is mathematically equivalent to multiplying the positive values of $D_{ij}$ by two and setting the negative ones to zero (because $\vec \sigma_i \times \vec \sigma_j = - \vec \sigma_j \times \vec \sigma_i$). 

I will take this second approach to avoid inconsistencies in the external fields, where we can have situations where site $i$ has an upper-right neighbor $j$ outside of the cluster, while site $j$ within the cluster has an upper-right neighbor $i$ out of the cluster. This would suggest $D_{ij}=1$ and $D_{ji}=1$, which is not what we would like from the antisymmetry of these coefficients. In the case of our 12-site cluster, this can be seen between sites 10 and 2 (among others).

                <4>-<5>
                / \ / \
          <9>-<0>-<1>-<2>-<7>
            \ / \ / \ / \ /
        <5>-<6>- a - b -<3>-<4>
          \ / \ / \ / \ / \ /
      <1>-<2>- 7 - 8 - 9 -<0>-<1>
        \ / \ / \ / \ / \ / \ /
    <a>-<b>- 3 - 4 - 5 - 6 -<a>-<b>
      \ / \ / \ / \ / \ / \ / \ /
      <8>-<9>- 0 - 1 - 2 -<7>-<8>
          / \ / \ / \ / \ / \
        <5>-<6>-<a>-<b>-<3>-<4>
              \ / \ / \ /
              <7>-<8>-<9>



In [ ]:
# Lists of bonds in particular directions within the cluster
up_right_in = [[0,5], [1,6], [3,8], [4,9], [7,11]]
up_left_in = [[1,3], [2,4], [5,7], [6,8], [9,10]]
down_in = [[7,0], [8,1], [9,2], [10,4], [11,5]]
in_inds = up_right_in + up_left_in + down_in

# Lists of bonds in same directions outside of the cluster
up_right_out = [[2,10], [5,0], [6,1], [8,3], [9,4], [10,2], [11,7]] # bond 10-2 shows up both directions
up_left_out = [[0,11], [3,1], [4,2], [7,5], [8,6], [10,9], [11,0]] # bond 11-0 shows up both directions
down_out = [[0,7], [1,8], [2,9], [3,6], [4,10], [5,11], [6,3]] # bond 3-6 shows up both directions
out_inds = up_right_out + up_left_out + down_out

# Now populating the d_{ij} matrices to have entries 1 for the bonds, 0 else

d_next_nearest_in = np.zeros((12, 12))
for ind in in_inds:
    i, j = ind
    d_next_nearest_in[i, j] = 1
    
d_next_nearest_out = np.zeros((12, 12))
for ind in out_inds:
    i, j = ind
    d_next_nearest_out[i,j] = 1

Now that I've constructed these coefficient matrices, I can use them to build Hamiltonians. My code can take a matrix of coefficients connecting individual sites, referred to as "coeffs" within the code itself. These follow a similar structure to the interactions, although with another level to specify whether they apply within or outside of the cluster, as is clearly important for the next-nearest-neighbor coefficients formed above.

I will also make the same sort of matrix for the nearest-neighbor interactions, so that everything is constructed similarly. Luckily, I already have the bonds in the code:

In [ ]:
in_inds = plaq12['inner']['x_bonds'] + plaq12['inner']['y_bonds'] + plaq12['inner']['z_bonds']
out_inds = plaq12['outer']['x_bonds'] + plaq12['outer']['y_bonds'] + plaq12['outer']['z_bonds']

d_nearest_in = np.zeros((12, 12))
for ind in in_inds:
    i, j = ind
    d_nearest_in[i, j] = 1
    
d_nearest_out = np.zeros((12, 12))
for ind in out_inds:
    i, j = ind
    d_nearest_out[i,j] = 1

Now, let's construct the Hamiltonian
\begin{equation}
H = D_1 \sum_{i,j \in \vartriangle} \left(\vec\sigma_i \times \vec\sigma_j\right)_z
+ D_2 \sum_{i,j \in \triangleright} \left(\vec\sigma_i \times \vec\sigma_j\right)_z
\end{equation}
where $\vartriangle$ contains the CCW nearest-neighbor upright triangle bonds and 
$\triangleright$ contains the CCW next-nearest-neighbor right-facing triangle bonds.

First, let's recreate the nearest-neighbor case to make sure nothing obvious is wrong. We should get the same energy as before when $D_2=0$:

In [ ]:
D1 = 1
D2 = 0
interactions = {'nearest': {'xy': D1, 'yx': -D1},
                'n_nearest': {'xy': D2, 'yx': -D2}}
coeffs = {'inner': {'nearest': {'xy': d_nearest_in, 'yx': d_nearest_in},
                    'n_nearest': {'xy': d_next_nearest_in, 'yx': d_next_nearest_in}},
          'outer': {'nearest': {'xy': d_nearest_out, 'yx': d_nearest_out},
                    'n_nearest': {'xy': d_next_nearest_out, 'yx': d_next_nearest_out}}
           }
Hi = hmf.operators.inner_hamiltonian(plaq12, interactions, basis, coeffs=coeffs)
e, v = Hi.eigsh(k=1, which='SA')
print('Energy with OBC: {}'.format(e[0]))

Hp = hmf.operators.periodic_hamiltonian(plaq12, interactions, basis, coeffs=coeffs)
e, v = Hp.eigsh(k=1, which='SA')
print('Energy with PBC: {}'.format(e[0]))

e_hmft, v, mf, cvg = hmf.do_hmft(plaq12, interactions, basis, coeffs=coeffs)

print('HMFT energy: {}'.format(e_hmft))
print('HMFT converged? {}'.format(cvg))

Now, we can add $D_2 \neq 0$:

In [ ]:
D1 = 1
D2 = .5
interactions = {'nearest': {'xy': D1, 'yx': -D1},
                'n_nearest': {'xy': D2, 'yx': -D2}}
coeffs = {'inner': {'nearest': {'xy': d_nearest_in, 'yx': d_nearest_in},
                    'n_nearest': {'xy': d_next_nearest_in, 'yx': d_next_nearest_in}},
          'outer': {'nearest': {'xy': d_nearest_out, 'yx': d_nearest_out},
                    'n_nearest': {'xy': d_next_nearest_out, 'yx': d_next_nearest_out}}
           }
Hi = hmf.operators.inner_hamiltonian(plaq12, interactions, basis, coeffs=coeffs)
e, v = Hi.eigsh(k=1, which='SA')
print('Energy with OBC: {}'.format(e[0]))

Hp = hmf.operators.periodic_hamiltonian(plaq12, interactions, basis, coeffs=coeffs)
e, v = Hp.eigsh(k=1, which='SA')
print('Energy with PBC: {}'.format(e[0]))

e_hmft, v, mf, cvg = hmf.do_hmft(plaq12, interactions, basis, coeffs=coeffs)

print('HMFT energy: {}'.format(e_hmft))
print('HMFT converged? {}'.format(cvg))

# Further neighbors
If we wish to include further neighbors, they can be included using the same method I used for the next-nearest neighbors. The next-next-nearest neighbors again form triangular sublattices, but there are four (rather than 3) and they are aligned in the same way as the original lattice.